In [2]:
import pandas as pd
import numpy as np


train = pd.read_csv("./Data/train_2016_v2.csv")    # load dataset
properties = pd.read_csv("./Data/properties_2016.csv")
test_df = pd.read_csv("./Data/sample_submission.csv")
test_df = test_df.rename(columns={'ParcelId': 'parcelid'})

/Users/rubychiu/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
train_df = train.merge(properties, how = 'left', on = 'parcelid') # get a complete dataframe
test_df = test_df.merge(properties, on='parcelid', how='left')


In [4]:
train_df.shape

(90275, 60)

In [5]:
# get outlier of logerror
ulimit = np.percentile(train_df.logerror.values, 99)  
print(ulimit)
llimit = np.percentile(train_df.logerror.values, 1)
print(llimit)

0.4638819999999963
-0.3425


In [6]:
# get train data and test data without outlier
# don't change the code about test data
train_df = train_df[train_df.logerror > -0.3425]
train_df = train_df[train_df.logerror < 0.4639]
do_not_include = ['parcelid', 'logerror', 'transactiondate']
feature_names = [f for f in train_df.columns if f not in do_not_include]
#X_train = train_df.drop(['parcelid', 'logerror', 'transactiondate'] ,axis = 1)
X_train = train_df[feature_names].copy()
y_train = train_df.logerror
test_df = test_df[feature_names].copy()


In [7]:
from sklearn.model_selection import train_test_split

# get training data and validation data(we don't need do this when we want to submit to kaggle, just use all data to train)

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, train_size =0.8, test_size = 0.2, random_state = 0)


In [8]:
# get the data index which is categorical 

s = (X_train.dtypes =='object')
object_cols = list(s[s].index)
print(object_cols)

['hashottuborspa', 'propertycountylandusecode', 'propertyzoningdesc', 'fireplaceflag', 'taxdelinquencyflag']


In [9]:
from sklearn.preprocessing import LabelEncoder


# use labelencoder to encoder X_train and test data
label_X_train = X_train.copy()
label_X_valid = X_valid.copy()
test = test_df.copy()
for col in label_X_train.columns:
    label_X_train[col] = label_X_train[col].fillna(-1)
    test[col] = test[col].fillna(-1)
    if label_X_train[col].dtype =='object':
        lbl = LabelEncoder()
        lbl.fit(list(label_X_train[col].values))
        label_X_train[col] = lbl.transform(list(label_X_train[col].values))
    if  test[col].dtype == 'object':
        lbl.fit(list(test[col].values))
        test[col] = lbl.transform(list(test[col].values))
x_test = test.copy()

#use labelencoder to encoder X_vaid, this part can be deleted
for col in label_X_valid.columns:
    label_X_valid[col] = label_X_valid[col].fillna(-1)
    if label_X_valid[col].dtype =='object':
        lbl = LabelEncoder()
        lbl.fit(list(label_X_valid[col].values))
        label_X_valid[col] = lbl.transform(list(label_X_valid[col].values))



In [10]:
print(label_X_train.shape)
y_train.shape

(70772, 57)


(70772,)

In [11]:
# XGBoost model 1

from xgboost import XGBRegressor

XGB_model_1 = XGBRegressor(n_estimators = 500, early_stopping_rounds = 5, learning_rate = 0.05, max_depth = 7, objective='reg:linear')
XGB_model_1.fit(label_X_train, y_train)

[20:09:57] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/Users/rubychiu/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, early_stopping_rounds=5,
             gamma=0, importance_type='gain', learning_rate=0.05,
             max_delta_step=0, max_depth=7, min_child_weight=1, missing=None,
             n_estimators=500, n_jobs=1, nthread=None, objective='reg:linear',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=1, verbosity=1)

In [12]:
from sklearn.metrics import mean_absolute_error

# get kaggle verification predictioin result
XGB_predictions_1 = XGB_model_1.predict(test)


In [56]:
XGB_predictions_1.shape


array([-0.03455275, -0.03566307,  0.03666675, ...,  0.2911566 ,
        0.2911566 ,  0.2911566 ], dtype=float32)

In [ ]:
# pd.DataFrame(XGB_predictions_1).head()

In [ ]:
# import matplotlib.pyplot as plt
# import xgboost

# fig, ax = plt.subplots(figsize=(12,18))
# xgboost.plot_importance(XGB_model_1, max_num_features=50, height=0.8, ax=ax)
# plt.show()

In [ ]:
# mae_XGB_1 = mean_absolute_error(y_valid, XGB_predictions_1)
# print(mae_XGB_1)

In [ ]:
# # get feature which importance score is higher than 600 from the picture
# #selected_features = ['taxamount', 'calculatedfinishedsquarefeet', 'structuretaxvaluedollarcnt', 'longitude',
#                                      'finishedsquarefeet12', 'latitude','regionidzip','taxvaluedollarcnt','lotsizesquarefeet',
#                                     'landtaxvaluedollarcnt','rawcensustractandblock','yearbuilt','propertyzoningdesc'
#                                     ,'taxdelinquencyyear','bathroomcnt','bedroomcnt','regionidneighborhood' ]

In [17]:
# writing csv result file
sub = pd.read_csv('../input/sample_submission.csv')
for c in sub.columns[sub.columns != 'ParcelId']:
    sub[c] = XGB_predictions_1 

print('Writing csv ...')
sub.to_csv('XGB_submission.csv', index=False, float_format='%.4f')

Writing csv ...


In [48]:
from sklearn.linear_model import Lasso
lasso = Lasso(alpha=0.3,max_iter=10000)
lasso.fit(label_X_train,y_train)


Lasso(alpha=0.5, copy_X=True, fit_intercept=True, max_iter=10000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [49]:
lasso_predictions = lasso.predict(test)
#test
#MAE_value = mean_absolute_error(lasso_predictions, y_valid)

In [50]:
# writing csv result file
sub = pd.read_csv('../input/Data/sample_submission.csv')
for c in sub.columns[sub.columns != 'ParcelId']:
    sub[c] = lasso_predictions
print('Writing csv ...')
sub.to_csv('Lasso_submission.csv', index=False, float_format='%.4f')

Writing csv ...
